In [133]:
import os
import pandas as pd

path = 'Attachments'
ids_list = []
ids_comment = {}
all_list = []

def parse_annotation(annot):
    pass

def compare_annotations(annot):
    pass

def parse_dict(all_list):
    pass
                
appended_annots = []
for filename in os.listdir(path):
    ids_taste = []
    list_ids_taste = []
    full_filename = os.path.join(path, filename)
    data = pd.read_excel(full_filename)      
    if 'id' in data.columns: #Mandarin files do not have id's
        appended_annots.append(data)
#print (appended_annots)        

all_frame = pd.concat(appended_annots)
#print ( all_frame.duplicated('id') )

#if all_frame.duplicated('id'):
#new_frame = 
print ("this is duplicate: ", all_frame[all_frame.duplicated('id', keep=False)] )






this is duplicate:  0      True
1      True
2      True
3     False
4     False
5      True
6      True
7     False
8      True
9      True
10    False
11     True
12     True
13     True
14    False
15     True
16    False
17    False
18     True
19     True
0      True
1      True
2     False
3      True
4      True
5      True
6      True
7     False
8     False
9      True
      ...  
10     True
11     True
12     True
13     True
14    False
15     True
16     True
17     True
18    False
19    False
0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
Length: 860, dtype: bool
